# **Getting started**  

setting up the required data files location from google drive 

In [441]:
#lodaing files folder from google drive 
#give file path in drive.mount('file path')
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [442]:
#importing libraries 
import nltk
import os
import re
import numpy as np
import pandas as pd
from nltk.corpus import PlaintextCorpusReader,stopwords

# ***extracting data into data frame***

In [443]:
import pandas as pd
df_csv=pd.read_csv("/content/drive/MyDrive/workingWithP/master_clauses_1.csv")
df_csv.head()

,Filename,Document Name,Document Name-Answer,Parties,Parties-Answer,Agreement Date,Agreement Date-Answer,Effective Date,Effective Date-Answer,Expiration Date,Expiration Date-Answer,Renewal Term,Renewal Term-Answer,Notice Period To Terminate Renewal,Notice Period To Terminate Renewal- Answer,Governing Law,Governing Law-Answer,Most Favored Nation,Most Favored Nation-Answer,Competitive Restriction Exception,Competitive Restriction Exception-Answer,Non-Compete,Non-Compete-Answer,Exclusivity,Exclusivity-Answer,No-Solicit Of Customers,No-Solicit Of Customers-Answer,No-Solicit Of Employees,No-Solicit Of Employees-Answer,Non-Disparagement,Non-Disparagement-Answer,Termination For Convenience,Termination For Convenience-Answer,Rofr/Rofo/Rofn,Rofr/Rofo/Rofn-Answer,Change Of Control,Change Of Control-Answer,Anti-Assignment,Anti-Assignment-Answer,Revenue/Profit Sharing,...,Minimum Commitment,Minimum Commitment-Answer,Volume Restriction,Volume Restriction-Answer,Ip Ownership Assignment,Ip Ownership Assignment-Answer,Joint Ip Ownership,Joint Ip Ownership-Answer,License Grant,License Grant-Answer,Non-Transferable License,Non-Transferable License-Answer,Affiliate License-Licensor,Affiliate License-Licensor-Answer,Affiliate License-Licensee,Affiliate License-Licensee-Answer,Unlimited/All-You-Can-Eat-License,Unlimited/All-You-Can-Eat-License-Answer,Irrevocable Or Perpetual License,Irrevocable Or Perpetual License-Answer,Source Code Escrow,Source Code Escrow-Answer,Post-Termination Services,Post-Termination Services-Answer,Audit Rights,Audit Rights-Answer,Uncapped Liability,Uncapped Liability-Answer,Cap On Liability,Cap On Liability-Answer,Liquidated Damages,Liquidated Damages-Answer,Warranty Duration,Warranty Duration-Answer,Insurance,Insurance-Answer,Covenant Not To Sue,Covenant Not To Sue-Answer,Third Party Beneficiary,Third Party Beneficiary-Answer
0,2ThemartComInc_19990826_10-12G_EX-10.10_670028...,['CO-BRANDING AND ADVERTISING AGREEMENT'],CO-BRANDING AND ADVERTISING AGREEMENT,"['2THEMART.COM, INC.', '2TheMart', 'i-Escrow',...","I-ESCROW, INC. (""i-Escrow"" ); 2THEMART.COM, I...","['June 21, 1999']",6/21/99,"['June 21, 1999']",6/21/99,['The term of this Agreement shall continue fo...,6/21/00,['In the event that either party does not give...,Successive 1 year,['In the event that either party does not give...,30 days,['This Agreement will be governed and construe...,California,[],No,[],No,"[""i-Escrow shall not run banner advertisements...",Yes,[],No,[],No,[],No,[],No,[],No,[],No,"[""If a majority of the equity securities of ei...",Yes,['All rights (under any applicable intellectua...,Yes,"['After the Launch Date, i-Escrow shall pay 2T...",...,[],No,[],No,[],No,"['Notwithstanding the foregoing, to the extent...",Yes,['Subject to the terms and conditions of this ...,Yes,['Subject to the terms and conditions of this ...,Yes,[],No,[],No,[],No,[],No,[],No,"['Notwithstanding the foregoing, unless this A...",Yes,"[""Such inspection shall be at 2TheMart's expen...",Yes,['EXCEPT IN THE EVENT OF A BREACH OF SECTION 1...,Yes,['EXCEPT IN THE EVENT OF A BREACH OF SECTION 1...,Yes,[],No,[],No,[],No,[],No,[],No
1,ABILITYINC_06_15_2020-EX-4.25-SERVICES AGREEME...,['Services Agreement'],Services Agreement,"['""Provider""', 'TELCOSTAR PTE, LTD.', 'Each of...","[ * * * ] (""Provider""); TELCOSTAR PTE, LTD.; A...","['October 1, 2019']",10/01/2019,"['November 1, 2019']",11/01/2019,['This Agreement be deemed effective as of the...,12/31/20,[],NaN,[],NaN,"['This Agreement and any claim, controversy or...",Israel,[],No,[],No,[],No,[],No,[],No,[],No,[],No,"['Each of the Recipient and the Provider may, ...",Yes,[],No,[],No,"['Provider may not assign, delegate or otherwi...",Yes,[],...,[],No,[],No,['To the extent that any writings or works of ...,Yes,[],No,[],No,[],No,[],No,[],No,[],No,[],No,[],No,[],No,[],No,[],No,[],No,[],No,[],No,[],No,[],No,[],No
2,ACCELERATEDTECHNOLOGIESHOLDINGCORP_04_24_2003-...,['JOINT VENTURE AGREEMENT'],JOINT VENTURE AGREEMENT,"['Pivotal Self 

In [444]:
#filtering requried information from extracted data that is text content and label
df = df_csv.filter(['Audit Rights','Audit Rights-Answer'], axis = 1)
df['label'] = df_csv['Audit Rights-Answer']
df['text'] = df_csv['Audit Rights']

**Text Pre-Processing**

In [445]:
#Importing Libraries
from bs4 import BeautifulSoup
import nltk

In [446]:
#general text processing 
# removing punctation and white spaces 
#setting the text to lower case
def text_preprocess(text):
  text=text.lower()
  text=re.compile('[/(){}\[\]\|@,;]').sub(' ', text)
  return text

In [447]:
df['text']=df['text'].apply(text_preprocess)
df['text'].head()

0     "such inspection shall be at 2themart's expen...
1                                                     
2     'all books and records of every kind and char...
3                                                     
4                                                     
Name: text, dtype: object

*punctuation removal*

In [448]:
# importing library that has punctuation
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [449]:
#defining a function to remove punctuation 
def remove_all_punctuation(text):
  punctuation_remove= "".join([i for i in text if i not in string.punctuation])
  return punctuation_remove


In [450]:
#calling punctuation_remove function to clean the data 
df['text'] = df['text'].apply(lambda x:remove_all_punctuation(x))
df['text']

0       such inspection shall be at 2themarts expense...
1                                                       
2       all books and records of every kind and chara...
3                                                       
4                                                       
                             ...                        
505                                                     
506     zogenix will have the right  upon reasonable ...
507                                                     
508     the schedule will be provided in accordance w...
509                                                     
Name: text, Length: 510, dtype: object

In [451]:
#space removel
df['text'] = df['text'].str.replace('\d+', '')

*lowering the text*

In [452]:
#setting the text to lower case
df['text'] = df['text'].apply(lambda x: x.lower())

*HTML removing*

In [453]:
#importing package from lib
from bs4 import BeautifulSoup 

In [454]:
#appling beautiful soup on the text 
df['text'] = [BeautifulSoup(str(text)).get_text() for text in df['text'].tolist()]

*unicode removal*

In [455]:
#imorting library 
import unicodedata

In [456]:
#appling unicode on the text data
df['text'] = [unicodedata.normalize('NFKD', str(text)).encode('ascii', 'ignore').decode('utf-8', 'ignore') for text in df['text'].tolist()]

contractions removing

In [457]:
#installing contraction library
!pip install contractions

In [458]:
#importing library
import contractions
from contractions import contractions_dict

In [459]:
#appling the contactions list on the text data to remove all contractions
for contraction, expansion in contractions_dict.items():
  df['text'] = df['text'].str.replace(contraction, expansion)

*tokenization*

In [460]:
#importing library for tokenization
import re
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [461]:
#defining function for tokenization
def tokenization(text):
  tokens = word_tokenize(text)
  return tokens

In [462]:
#calling tokenization function 
df['text'] = df['text'].apply(str)
df['text'] = df['text'].apply(lambda x: tokenization(x))
df['text']

0      [such, inspection, shall, be, at, ththemarchs,...
1                                                     []
2      [all, books, and, records, of, every, kind, an...
3                                                     []
4                                                     []
                             ...                        
505                                                   []
506    [zogenix, will, have, the, right, upon, reason...
507                                                   []
508    [the, schedule, will, be, provided, in, accord...
509                                                   []
Name: text, Length: 510, dtype: object

*stop word removal*

In [463]:
# nltk lib has already imported 
#stop words in given library
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:20]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

In [464]:
#defining function for stop words
def remove_stopwords(text):
  output = [i for i in text if i not in stopwords]
  return output

In [465]:
#appling the stopwords function to the text content
df['text'] = df['text'].apply(lambda x:remove_stopwords(x))
df['text']

0      [inspection, shall, ththemarchs, expense, howe...
1                                                     []
2      [books, records, every, kind, character, joint...
3                                                     []
4                                                     []
                             ...                        
505                                                   []
506    [zogenix, right, upon, reasonable, prior, writ...
507                                                   []
508    [schedule, provided, accordance, requirthement...
509                                                   []
Name: text, Length: 510, dtype: object

*lemmatization*

In [466]:
#importing lemmatization from lib
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [467]:
#defining functioon as object
wordnet_lemmatizer = WordNetLemmatizer()

In [468]:
#defining the lemmatization function
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text

In [469]:
#appling lemmization function to text content
df['text'] = df['text'].apply(lambda x: lemmatizer(x))

*null values removing*

In [470]:
# removing white spaces or null values from the text content 
df = df[~(df.text.str.strip() == '')]
df = df.dropna().reset_index(drop=True)

# **splitting data**

splitting corpus into two parts for testing and training the model.
training - 80% 
test - 20 % 
so the corpus has to spilt in the ratio of 80:20.


In [471]:
from sklearn.model_selection import train_test_split

In [472]:
#dividing up labels and data for testing as well as training
train_corpus_set, test_corpus_set, train_label_names_set, test_label_names_set = train_test_split(np.array(df['text'].apply(lambda x:np.str_(x))),
np.array(df['label']), 
test_size=0.20, 
random_state=42)

train_corpus_set.shape, test_corpus_set.shape

((408,), (102,))

# **TF-IDF and classification models**

## **features extraction**

## *TF-IDF* 

In [473]:
#importing tfidf package from skleran
from sklearn.feature_extraction.text import TfidfVectorizer

In [474]:
#normilization of TF-IDF algorithm
tfidf_tv = TfidfVectorizer(use_idf=True, min_df=0.0, max_df=1.0)

In [475]:
#appling the TF-IDF algorithm to extract features
tfidf_tv_train_features = tfidf_tv.fit_transform(train_corpus_set)
tfidf_tv_test_features = tfidf_tv.transform(test_corpus_set)

In [476]:
print('TFIDF model:\n','Train features shape:', tfidf_tv_train_features.shape,'\n', ' Test features shape:', tfidf_tv_test_features.shape)

TFIDF model:
 Train features shape: (408, 1762) 
  Test features shape: (102, 1762)


## **classification model and  evaluation**

## *SVM*

In [477]:
# importing SVM algorithm from sklearn library
from sklearn.svm import LinearSVC

In [478]:
#applying the SVM algorithm on training data to train the model
svm_model = LinearSVC(penalty='l2', C=1, random_state=42)
svm_model.fit(tfidf_tv_train_features, train_label_names_set)

LinearSVC(C=1, random_state=42)

In [479]:
#classification report of SVM Model
from sklearn.metrics import classification_report
y_pred = svm_model.predict(tfidf_tv_test_features)
print(classification_report(test_label_names_set, y_pred))

              precision    recall  f1-score   support

          No       1.00      1.00      1.00        63
         Yes       1.00      1.00      1.00        39

    accuracy                           1.00       102
   macro avg       1.00      1.00      1.00       102
weighted avg       1.00      1.00      1.00       102



###confusion matrix

In [480]:
#confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix_predict = confusion_matrix(test_label_names_set , y_pred)
print(confusion_matrix_predict)

[[63  0]
 [ 0 39]]


## *logestic regression*

In [481]:
#impoorting logistic regression algorithm from sklearn library
import sklearn.linear_model as sk
from sklearn.linear_model import LogisticRegression


In [482]:
#applying the SVM algorithm on training data to train the model
lr_model = LogisticRegression(penalty='l2', solver='lbfgs', multi_class='ovr',
                        max_iter=1000, C=1, random_state=42)
lr_model.fit(tfidf_tv_train_features, train_label_names_set)

LogisticRegression(C=1, max_iter=1000, multi_class='ovr', random_state=42)

In [483]:
#testing and classification report of LOgestic regression
y_pred = lr_model.predict(tfidf_tv_test_features)
print(classification_report(test_label_names_set, y_pred))

              precision    recall  f1-score   support

          No       0.97      1.00      0.98        63
         Yes       1.00      0.95      0.97        39

    accuracy                           0.98       102
   macro avg       0.98      0.97      0.98       102
weighted avg       0.98      0.98      0.98       102



###confusion matrix

In [484]:
#confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix_predict = confusion_matrix(test_label_names_set , y_pred)
print(confusion_matrix_predict)

[[63  0]
 [ 2 37]]
